# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [3]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [15]:
data.head(20)

CustomerID  FirstName   LastName  SalesID  ProductID  \
0        61288       Rosa   Andersen   134196        229   
1        77352      Myron     Murray  6167892        229   
2        40094      Susan  Stevenson  5970885        229   
3        23548     Tricia    Vincent  6426954        229   
4        78981      Scott      Burch   819094        229   
5        83106       Hope     Bowers  6484922        229   
6        11253    Nicolas     Holmes  4878478        229   
7        35107     Judith    Miranda  2353899        229   
8        15088       Ruby       Lowe    37021        229   
9        26031  Josephine      Curry  2327232        229   
10       26031  Josephine      Curry  2634983        229   
11       86595      Emily       Huff   248932        229   
12       46715     Angelo    Ballard  1630185        229   
13       91013       Toni     Hayden  3952547        229   
14       69931     Jennie     Bishop  4638141        229   
15       19753      Alana    Carroll  4143712        229   
16        9546     Dianna      Cline    62517        229   
17       10224     Stacie     Larson   436036        229   
18       72846    Carlton      Cowan  5428647        229   
19       28976  Gabrielle    Vaughan  4404856        229   

             ProductName  Quantity  
0   Bread - Hot Dog Buns        16  
1   Bread - Hot Dog Buns        20  
2   Bread - Hot Dog Buns        11  
3   Bread - Hot Dog Buns         6  
4   Bread - Hot Dog Buns        20  
5   Bread - Hot Dog Buns        22  
6   Bread - Hot Dog Buns         3  
7   Bread - Hot Dog Buns         9  
8   Bread - Hot Dog Buns         4  
9   Bread - Hot Dog Buns         7  
10  Bread - Hot Dog Buns         7  
11  Bread - Hot Dog Buns        22  
12  Bread - Hot Dog Buns        12  
13  Bread - Hot Dog Buns        24  
14  Bread - Hot Dog Buns        18  
15  Bread - Hot Dog Buns         6  
16  Bread - Hot Dog Buns         3  
17  Bread - Hot Dog Buns         3  
18  Bread - Hot Dog Buns        19  
19  Bread - Hot Dog Buns         8

## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [37]:

df = data.groupby(['CustomerID','ProductName']).agg({'Quantity':'sum'})
df.head(10)

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1
           Beef - Chuck, Boneless                1
           Beef - Prime Rib Aaa                  1
           Beer - Original Organic Lager         1
           Beer - Rickards Red                   1
           Black Currants                        1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [38]:
tabla = df.pivot_table(values='Quantity',columns='CustomerID', index='ProductName',aggfunc='sum').fillna(0)


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [40]:
squareform(pdist(tabla.T, 'euclidean'))

array([[  0.        ,  11.91637529,  10.48808848, ..., 228.62851966,
        239.        , 229.77380181],
       [ 11.91637529,   0.        ,  11.74734012, ..., 228.01096465,
        239.03765394, 229.70415756],
       [ 10.48808848,  11.74734012,   0.        , ..., 228.08112592,
        238.26665734, 229.77380181],
       ...,
       [228.62851966, 228.01096465, 228.08112592, ...,   0.        ,
        304.13812651, 305.16389039],
       [239.        , 239.03765394, 238.26665734, ..., 304.13812651,
          0.        , 303.10889132],
       [229.77380181, 229.70415756, 229.77380181, ..., 305.16389039,
        303.10889132,   0.        ]])

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [49]:
a = pd.DataFrame(1/(1 + squareform(pdist(tabla.T, 'euclidean'))), index=tabla.columns, columns=tabla.columns)
a.head()
def lista(id):
    similarities = a[id].sort_values(ascending=False)[1:]
    return similarities.head()

lista(33)

CustomerID
264     0.087047
3535    0.087047
3317    0.087047
2503    0.085983
3305    0.085638
Name: 33, dtype: float64


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [52]:
c=df.loc[lista(33).index]
c

Quantity
CustomerID ProductName                             
264        Apricots - Halves                      1
           Apricots Fresh                         1
           Bacardi Breezer - Tropical             1
           Bagel - Plain                          1
           Banana - Leaves                        1
...                                             ...
3535       Tray - 16in Rnd Blk                    1
           Wanton Wrap                            1
           Wine - Blue Nun Qualitatswein          1
           Wine - Magnotta, Merlot Sr Vqa         1
           Yoghurt Tubes                          1

[270 rows x 1 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [58]:
b = c.groupby('ProductName').Quantity.sum()
b = b.sort_values(ascending=False)
b

ProductName
Butter - Unsalted                3
Soup - Campbells Bean Medley     3
Towels - Paper / Kraft           3
Wine - Ej Gallo Sierra Valley    3
Pepper - Black, Whole            3
                                ..
Pepper - White, Ground           1
Pepper - Paprika, Hungarian      1
Pate - Cognac                    1
Pastry - Choclate Baked          1
Anchovy Paste - 56 G Tube        1
Name: Quantity, Length: 218, dtype: int64

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [66]:
listap = df.loc[33]
rank =b.sort_values(ascending=False)
miss =[x for x in rank.index if x not in listap.index]
rank.head()

ProductName
Butter - Unsalted                3
Soup - Campbells Bean Medley     3
Towels - Paper / Kraft           3
Wine - Ej Gallo Sierra Valley    3
Pepper - Black, Whole            3
Name: Quantity, dtype: int64

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [71]:
customer = set(data['CustomerID'].values)
distancia = pd.DataFrame(1/(1 + squareform(pdist(tabla.T, 'euclidean'))), index=tabla.columns, columns=tabla.columns)
distancia

CustomerID     33        200       264       356       412       464    \
CustomerID                                                               
33          1.000000  0.077421  0.087047  0.081800  0.080634  0.082709   
200         0.077421  1.000000  0.078448  0.076435  0.073693  0.075255   
264         0.087047  0.078448  1.000000  0.080070  0.081800  0.080350   
356         0.081800  0.076435  0.080070  1.000000  0.076435  0.078187   
412         0.080634  0.073693  0.081800  0.076435  1.000000  0.078711   
...              ...       ...       ...       ...       ...       ...   
97928       0.004515  0.004521  0.004543  0.004526  0.004530  0.004533   
98069       0.004583  0.004614  0.004595  0.004578  0.004578  0.004578   
98159       0.004355  0.004367  0.004365  0.004365  0.004367  0.004363   
98185       0.004167  0.004166  0.004179  0.004175  0.004177  0.004167   
98200       0.004333  0.004335  0.004333  0.004339  0.004349  0.004347   

CustomerID     477       639       649       669    ...     97697     97753  \
CustomerID                                          ...                       
33          0.074573  0.083020  0.081503  0.080070  ...  0.004811  0.004669   
200         0.075956  0.076435  0.077674  0.076923  ...  0.004824  0.004681   
264         0.076923  0.080634  0.082100  0.078448  ...  0.004822  0.004674   
356         0.075025  0.082403  0.077171  0.075956  ...  0.004816  0.004671   
412         0.075025  0.082403  0.078187  0.078448  ...  0.004810  0.004702   
...              ...       ...       ...       ...  ...       ...       ...   
97928       0.004534  0.004526  0.004528  0.004523  ...  0.003509  0.003509   
98069       0.004589  0.004581  0.004585  0.004592  ...  0.003509  0.003509   
98159       0.004366  0.004363  0.004369  0.004365  ...  0.003509  0.003565   
98185       0.004168  0.004176  0.004172  0.004161  ...  0.003311  0.003162   
98200       0.004338  0.004335  0.004329  0.004347  ...  0.003418  0.003345   

CustomerID     97769     97793     97900     97928     98069     98159  \
CustomerID                                                               
33          0.004412  0.005019  0.004312  0.004515  0.004583  0.004355   
200         0.004431  0.005047  0.004311  0.004521  0.004614  0.004367   
264         0.004416  0.005035  0.004322  0.004543  0.004595  0.004365   
356         0.004416  0.005038  0.004310  0.004526  0.004578  0.004365   
412         0.004414  0.005034  0.004318  0.004530  0.004578  0.004367   
...              ...       ...       ...       ...       ...       ...   
97928       0.003300  0.003653  0.003579  1.000000  0.003638  0.003418   
98069       0.003369  0.003509  0.003496  0.003638  1.000000  0.003469   
98159       0.003322  0.003431  0.003322  0.003418  0.003469  1.000000   
98185       0.003255  0.003406  0.003172  0.003255  0.003345  0.003277   
98200       0.003381  0.003444  0.003288  0.003245  0.003381  0.003266   

CustomerID     98185     98200  
CustomerID                      
33          0.004167  0.004333  
200         0.004166  0.004335  
264         0.004179  0.004333  
356         0.004175  0.004339  
412         0.004177  0.004349  
...              ...       ...  
97928       0.003255  0.003245  
98069       0.003345  0.003381  
98159       0.003277  0.003266  
98185       1.000000  0.003288  
98200       0.003288  1.000000  

[1000 rows x 1000 columns]

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [87]:
data.groupby(['CustomerID','ProductName']).head(5)


CustomerID FirstName   LastName  SalesID  ProductID  \
0           61288      Rosa   Andersen   134196        229   
1           77352     Myron     Murray  6167892        229   
2           40094     Susan  Stevenson  5970885        229   
3           23548    Tricia    Vincent  6426954        229   
4           78981     Scott      Burch   819094        229   
...           ...       ...        ...      ...        ...   
68579       94547    Gerald    Donovan   687884        211   
68580       57654  Lashonda        Roy  4373415        211   
68581       36751      Glen  Frederick  2405597        211   
68582       43270     Devon  Mc Daniel  4328852        211   
68583       78718     Mandy     Abbott  6175098        211   

                         ProductName  Quantity  
0               Bread - Hot Dog Buns        16  
1               Bread - Hot Dog Buns        20  
2               Bread - Hot Dog Buns        11  
3               Bread - Hot Dog Buns         6  
4               Bread - Hot Dog Buns        20  
...                              ...       ...  
68579  Wine - Prosecco Valdobiaddene        24  
68580  Wine - Prosecco Valdobiaddene        15  
68581  Wine - Prosecco Valdobiaddene        10  
68582  Wine - Prosecco Valdobiaddene        11  
68583  Wine - Prosecco Valdobiaddene        20  

[68584 rows x 7 columns]

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.